# Projecto Pump it Up: Data Mining the Water Table
realizado por: Christopher A.Rodriguez Principe


Usando datos de Taarifa y el Ministerio de Agua de Tanzania podemos predecir qué bombas funcionan, cuáles necesitan algunas reparaciones y cuáles no funcionan en absoluto.
Una comprensión inteligente de qué puntos de agua fallarán puede mejorar las operaciones de mantenimiento y garantizar que las comunidades de Tanzania dispongan de agua limpia y potable.

Este proyecto fue hecho en su totalidad por Christopher A.Rodriguez Principe. En este informe, estare discutiendo a profundidad las desiciones,observaciones y justificaciones que se hicieron durante la construccion del modelo predictivo.

In [1]:
#Importamos los packetes
import pandas as pd
from datetime import datetime
import plotly as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn import naive_bayes
from sklearn.model_selection import cross_val_score

from sklearn.metrics import roc_auc_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import accuracy_score, auc, confusion_matrix, f1_score, precision_score, recall_score, roc_curve
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression 
from sklearn.feature_selection import chi2 





### Importacion de Datos

Los datos fueron provistos por Taarifa y el ministro de agua de Tanzania. La variable denominada como 'Test_set_values' contiene las variables independientes cuyas queremos pasar por nuestro modelo para crear las predicciones.Las siguientes variables 'Training_set_labels' y 'Training_set_values' contienen las variables independientes y dependente con las que se entrenara el modelo.Estos dos dataframe los unimos en base al 'id', de esta manera el resultado se le asigna al id respectivo que le pertenece a las variables independientes. El dataframe resultante de esta union tiene 40 variables independientes + 1 variable dependiente(resultado).

In [2]:
#Cargamos los datos
Test_set_values= pd.read_csv('/Users/christopher/Desktop/DS UCM /Modulo 10/Competition/702ddfc5-68cd-4d1d-a0de-f5f566f76d91.csv')

Training_set_labels= pd.read_csv('/Users/christopher/Desktop/DS UCM /Modulo 10/Competition/0bf8bc6e-30d0-4c50-956a-603fc693d966.csv')
Training_set_values= pd.read_csv('/Users/christopher/Desktop/DS UCM /Modulo 10/Competition/4910797b-ee55-40a7-8668-10efd5c1b960.csv')


In [3]:
DF= pd.merge(Training_set_values,Training_set_labels,on=['id'],how='left')
print(DF.shape)

DF_test= Test_set_values

(59400, 41)


### Valores atipicos

Uno de los primeros pasos que demos hacer cuando nos enfrentamos con una serie de datos es familiarizarce con las variables. En la siguiente linea de codigo he hecho un .describe() para conocer nuestras variables numericas. De las primeras observaciones que podemos notar es que la variable 'num_private' tiene alrededor de 75% de sus valores son 0. Dado que no existe una definicion de la variable prestada por el ministro de agua de tanzania, decidi eliminar esta variable pensando que pudiese ser un error de sistema haber asignado ese numero. En aicion, he hecho las transformaciones tanto para el DF (dataframe training) como para DF_test(dataframe testing).

In [4]:
DF.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


In [5]:
def drop_num_amount(DF):
    #eliminamos variables que tienen más de 75% valores missing(0). 
    DF = DF.drop(['num_private'],1)
    return(DF)
    

DF= drop_num_amount(DF)
DF_test =drop_num_amount(DF_test)

/var/folders/cm/2vncb7d5433gcddk1v7h2mph0000gn/T/ipykernel_8563/4116403400.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  DF = DF.drop(['num_private'],1)


La siguiente transformacion se a he hecho a la variable 'construction_year'. Esto pues, alrededor de los datos eran 0. Esta variable representa el anio en que se construyo el pozo, pues no es logico que se construya en el anio 0. He remplazado los valores con la mediana (anio 1986). Se ha realizado un filtro al DF para eliminar las poblaciones con valores atipicos, nos concentramos en todas aquellas que tengan 600 habitantes o menos.

In [6]:
#filter for construction_year

DF.construction_year.replace((0),(1986),inplace=True)
DF_test.construction_year.replace((0),(1986),inplace=True)


#filter for construction_year
DF=DF[DF['population'] <= 600] 
DF.index=list(range(0,len(DF)))

In [7]:

print(DF.shape)
print(DF_test.shape)


(55984, 40)
(14850, 39)


### Changing Data types

Cambiamos el dtype de 'construction_year' y 'date_recorded' para crear variables dependientes del tiempo

In [8]:
DF.date_recorded = pd.to_datetime(DF.date_recorded,format="%Y-%m-%d")
DF.construction_year = pd.to_datetime(DF.construction_year,format='%Y')

DF.date_recorded = pd.DatetimeIndex(DF.date_recorded).year
DF.construction_year = pd.DatetimeIndex(DF.construction_year).year



### Data Test
DF_test.date_recorded = pd.to_datetime(DF_test.date_recorded,format="%Y-%m-%d")

DF_test.construction_year.replace((0),(1986),inplace=True)
DF_test.construction_year = pd.to_datetime(DF_test.construction_year,format='%Y')

DF_test.date_recorded = pd.DatetimeIndex(DF_test.date_recorded).year
DF_test.construction_year = pd.DatetimeIndex(DF_test.construction_year).year


### Eliminamos Variables con valores Unicos o Valores repetidos

La siguientes variables se han eliminado dado que enendí que muchas estaban repetidas y representaban la misma informacion. Tome esta desicion para simplificar los datos y quedarnos con las variable mas relevante y simplificada. Esta desicion usualmente se puede disctir con el cliente/persona del negocio que tiene conocimiento sobre las variables y su impacto. Dado que no existe un agente de negocio he eliminado as variables repetidas a mi interpretacion. He adicionalmente eliminado la variable 'recorded_by' dado que solamente tiene un valor y no aporta informacion al modelo.

In [9]:
#Eliminamos variable con valor unico
DF = DF.drop(['recorded_by'],1)
DF.index=list(range(0,len(DF)))

#Eliminar variables con valores repetidos
DF = DF.drop(['longitude','latitude','gps_height','funder','installer','wpt_name','basin','subvillage','region','lga','ward','extraction_type','scheme_name','extraction_type','extraction_type_group','management','payment_type','water_quality','quantity','source','waterpoint_type'],1)




### Data Test
DF_test = DF_test.drop(['recorded_by'],1)
DF_test.index=list(range(0,len(DF_test)))

#Eliminar variables con valores repetidos
DF_test = DF_test.drop(['longitude','latitude','gps_height','funder','installer','wpt_name','basin','subvillage','region','lga','ward','extraction_type','scheme_name','extraction_type','extraction_type_group','management','payment_type','water_quality','quantity','source','waterpoint_type'],1)




/var/folders/cm/2vncb7d5433gcddk1v7h2mph0000gn/T/ipykernel_8563/1994704689.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  DF = DF.drop(['recorded_by'],1)
/var/folders/cm/2vncb7d5433gcddk1v7h2mph0000gn/T/ipykernel_8563/1994704689.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  DF = DF.drop(['longitude','latitude','gps_height','funder','installer','wpt_name','basin','subvillage','region','lga','ward','extraction_type','scheme_name','extraction_type','extraction_type_group','management','payment_type','water_quality','quantity','source','waterpoint_type'],1)
/var/folders/cm/2vncb7d5433gcddk1v7h2mph0000gn/T/ipykernel_8563/1994704689.py:12: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  DF_test = DF_test.drop(['recorded_by'],1

In [10]:
print(DF.shape)
DF_test.shape

(55984, 19)


(14850, 18)

### Feature Engeneering

La siguiente estapa del projecto se dedica a la creacion de variables nuevas para implementar en el modelo. Las variables añadidas se han producido de ingenio sobre las variables existentes ya que no se puede importar datos de otra fuente para el proposito de la competencia. Habiendo dicho eso, siempre es importante preguntar a la persona de contacto de negocio por otras fuentes de datos que puedan ser incluidas para el desarrollo del modelo. 

He creado 4 funciones que producen variables distintas. De las 4 funciones he elegido unicamente la variable 'Years_since_observed2' dado que las otras variable no demostraron aumentar la abilidad predictiva del modelo. 
Las he incluido para proposito de refrencia.

La variable creada(Years_since_observed) se calcula restando la fecha en la que el pozo aparece en el sistema de registro y el año en que se construye el pozo. De esta forma podemos conocer cuantos años de funcion tiene el pozo. 

In [11]:
def Years_since_observed(DF):
    DF['Years_since_observed'] = (DF.date_recorded - DF.construction_year)
    DF.Years_since_observed = DF.Years_since_observed[DF.Years_since_observed >=0]
    DF = DF[DF['Years_since_observed'].notnull()]
    DF.Years_since_observed= DF.Years_since_observed.astype('int64')
    return(DF)

DF = Years_since_observed(DF)




def Years_since_observed2(DF):
    DF['Years_since_observed'] = (DF.date_recorded - DF.construction_year)

    return(DF)

DF_test = Years_since_observed2(DF_test)




/Users/christopher/opt/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


Las siguientes 3 funciones presentaban variables nuevas pero no aumentaron la capacidad predictiva del modelo. Esto se debe que en los datos de testing no existia la categoria en que se entreno los datos de entrenamiento. Especulo si existieran mas datos a test pudiera variar los datos para que sean similar a los datos de entrenamiento. 

In [12]:
def Y_times_recorded(DF):
    DF['Y_times_recorded']= DF.groupby('date_recorded')['date_recorded'].transform('count')
    DF.Y_times_recorded= DF.Y_times_recorded.astype('int64')
    DF.Y_times_recorded= DF['Y_times_recorded'].apply(lambda x:'high' if x>=9000 else('medium' if x<=8999 and x>=4500 else('low' if x<=4449 else(0))))
    return(DF)

In [13]:
def Num_bombas_Region(DF):
    DF['Num_bombas_Region']= DF.groupby('district_code')['district_code'].transform('count')
    DF.Num_bombas_Region= DF.Num_bombas_Region.astype('int64')
    DF.Num_bombas_Region= DF['Num_bombas_Region'].apply(lambda x:'high' if x>=3765 else('medium' if x<=3764 and x>=2895 else('low' if x<=2894 else(0))))
    return(DF)

In [14]:
def population(DF):
    DF.population= DF['population'].apply(lambda x:'high density' if x>=200 else('medium density' if x<=199 and x>=80 else('low density' if x<=79 else(0))))
    return(DF)

In [15]:
print(DF.shape)
DF_test.shape

(55977, 20)


(14850, 19)

### Convertimos a numerico las categorias

Muchos modelos de ML no acceptan valores categoricos como 'input'. He revisado las variables con tipos de dato 'object' y la he añadido a una lista. Estas variables se convertiran a numericas mediante One-hot encoding.
La funcion 'lista_categorias' nos crea estas variables. 

In [16]:
def lista_categorias(DF):
    ## Transformamos las variables a dummies
    lista_categoricas=['waterpoint_type_group','source_class','source_type','quantity_group','quality_group','payment','management_group','extraction_type_class','permit','scheme_management','public_meeting']
    for k in lista_categoricas:
        dummies = pd.get_dummies(DF[k],prefix=k,drop_first=True)
        DF = pd.concat([DF, dummies], axis=1)
        DF.drop(k,axis=1,inplace=True)
    return(DF)

DF = lista_categorias(DF)
DF_test= lista_categorias(DF_test)

Luego convertimos el resultado 'status_group' a numerico mediante label-encoding.

In [17]:
DF.status_group.replace(('functional','functional needs repair','non functional'),(0,1,2),inplace=True)

Elimino las variables:  'date_recorded','id','construction_year' del modelo ya que solo se utilizan para proposito de crear otras variables y modifica la base de datos.

In [18]:
DF= DF.drop(columns=['date_recorded','id','construction_year']) 

### 
DF_test_id = DF_test['id']
DF_test_id=pd.DataFrame(DF_test_id)

DF_test= DF_test.drop(columns=['date_recorded','id','construction_year']) 


La siguiente funcion nos consigue la correlacion entre variables. Para propositos de este proyecto he decidido eliminar una de las variables con correlacion >= 0.70. Para propositos fuera de la academia podemos considerar valores alrededor de 0.70 o consultar con la persona de negocio. 

In [19]:
df=DF

def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr(method='spearman').abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]


print("Top Absolute Correlations")
corrauto=(get_top_abs_correlations(df, 15))
print(corrauto)



Top Absolute Correlations
source_class_unknown                      source_type_other                  1.000000
waterpoint_type_group_hand pump           extraction_type_class_handpump     0.944711
management_group_parastatal               scheme_management_Parastatal       0.911332
source_class_surface                      source_type_river/lake             0.857594
waterpoint_type_group_communal standpipe  waterpoint_type_group_hand pump    0.763848
quality_group_good                        quality_group_salty                0.743111
waterpoint_type_group_communal standpipe  source_type_shallow well           0.740333
                                          extraction_type_class_handpump     0.725837
waterpoint_type_group_other               extraction_type_class_other        0.709624
quantity_group_enough                     quantity_group_insufficient        0.652821
waterpoint_type_group_hand pump           source_type_shallow well           0.647555
source_type_shallow well    

In [20]:
#Eliminamos variables correlacionadas con más de 0.70 colinealidad
DF =DF.drop(columns=['extraction_type_class_handpump','extraction_type_class_other','source_type_shallow well','source_type_river/lake','quality_group_salty','waterpoint_type_group_hand pump','scheme_management_Parastatal','source_type_other'])
DF_test =DF_test.drop(columns=['extraction_type_class_handpump','extraction_type_class_other','source_type_shallow well','source_type_river/lake','quality_group_salty','waterpoint_type_group_hand pump','scheme_management_Parastatal','source_type_other'])


In [21]:
DF = DF.astype('int64') 
DF_test = DF_test.astype('int64') 

### Feature Importance

La siguiente fase me dedico a decidir cuales variables tiene mayor importancia para predecir 'status_group'. Existen distintos metodos de seleccion de variable. He elegido elegir mediante SelectKBest que fue prestada en la documentacion del modulo. He elegido 25 columnas que mejor abilidad predictiva tendran. estas son: ['Years_since_observed', 'amount_tsh', 'district_code', 'extraction_type_class_motorpump', 'payment_pay annually', 'payment_pay monthly', 'payment_pay per bucket', 'payment_unknown', 'population', 'quality_group_good', 'quality_group_unknown', 'quantity_group_enough', 'quantity_group_unknown', 'region_code', 'scheme_management_VWC', 'scheme_management_WUA', 'scheme_management_WUG', 'scheme_management_Water Board', 'scheme_management_Water authority', 'source_class_surface', 'source_type_rainwater harvesting', 'source_type_spring', 'status_group', 'waterpoint_type_group_communal standpipe', 'waterpoint_type_group_other']. Podemos notar que la variable que he creado 'Years_since_observed' aparece entre las variables significativas. 

In [22]:
S_chi5 = SelectKBest(chi2, k = 25)
X_chi5 = S_chi5.fit_transform(DF, DF['status_group'])

variables = list(np.asarray(list(DF))[S_chi5.get_support()])
variables.sort()
print(variables)

['Years_since_observed', 'amount_tsh', 'district_code', 'extraction_type_class_motorpump', 'payment_pay annually', 'payment_pay monthly', 'payment_pay per bucket', 'payment_unknown', 'population', 'quality_group_good', 'quality_group_unknown', 'quantity_group_enough', 'quantity_group_unknown', 'region_code', 'scheme_management_VWC', 'scheme_management_WUA', 'scheme_management_WUG', 'scheme_management_Water Board', 'scheme_management_Water authority', 'source_class_surface', 'source_type_rainwater harvesting', 'source_type_spring', 'status_group', 'waterpoint_type_group_communal standpipe', 'waterpoint_type_group_other']


In [23]:
columnas_train = ['Years_since_observed', 'amount_tsh', 'district_code', 'extraction_type_class_motorpump', 'payment_pay annually', 'payment_pay monthly', 'payment_pay per bucket', 'payment_unknown', 'population', 'quality_group_good', 'quality_group_unknown', 'quantity_group_enough', 'quantity_group_unknown', 'region_code', 'scheme_management_VWC', 'scheme_management_WUA', 'scheme_management_WUG', 'scheme_management_Water Board', 'scheme_management_Water authority', 'source_class_surface', 'source_type_rainwater harvesting', 'source_type_spring','waterpoint_type_group_communal standpipe', 'waterpoint_type_group_other','status_group']
columnas_test =  ['Years_since_observed', 'amount_tsh', 'district_code', 'extraction_type_class_motorpump', 'payment_pay annually', 'payment_pay monthly', 'payment_pay per bucket', 'payment_unknown', 'population', 'quality_group_good', 'quality_group_unknown', 'quantity_group_enough', 'quantity_group_unknown', 'region_code', 'scheme_management_VWC', 'scheme_management_WUA', 'scheme_management_WUG', 'scheme_management_Water Board', 'scheme_management_Water authority', 'source_class_surface', 'source_type_rainwater harvesting', 'source_type_spring','waterpoint_type_group_communal standpipe', 'waterpoint_type_group_other',]

In [24]:
DF = DF[columnas_train]
DF_test = DF_test[columnas_test]


In [25]:
X = DF.iloc[:,:-1]
y = DF['status_group']


### Train/split model

Para proposito de nuestro data set he hecho varias pruebas para determinar el test_size de la funcion 'train_test_split'. He determinado elegir 20% de los datos para testing y 80% para hacer el entreamiento del modelo. Este porcentage lo he intendado con diferentes propociones y hacerlo con 20% me resulto con mejor capacidad predictiva para el modelo. 

In [26]:
from sklearn.model_selection import train_test_split
X_train,  X_test, y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

## Modelos

La siguiente estapa creamos varios modelos y probamos cual resulta con mayor Accuracy. Entre los modelos elegí: RandomForest, DecisionTreeCassifier, Naive Bayes, Linear Regresion. Quisiera enfatizar que por problemas con mi version de python, no he podido installar el modelo XGboost. Reconozco que este modelos es uno muy util y permite alzanzar mejor resultado pero dado las inconveniencias de mi sistema operativo lo he obviado. 

**Resultados de los modelos**: 
El modelo con mejor capacidad predictiva en el dataset de entrenamiento es Random Forest con un score de 0.76. Los prametros se han determinado de siguiente: n_estimators = 500. El numero de estimador se selecciono con varias puebas y 500 dio resultado a un mejor accuracy. He incluido el 'cross_val_score'  para determinar si nuestros datos estas over/under fitted, observamos que nuestro 'cross_val_score' se mantiene alrededor a 0.75 a la centesima . Por lo tanto determinamos que nuestros datos estan 'fitted' correctamente y procedemos con nuestro modelo. 

### RandomForest

In [27]:
random_forest= RandomForestClassifier(n_estimators=500,random_state=42)
random_forest.fit(X_train,y_train)

RandomForestClassifier(n_estimators=500, random_state=42)

In [28]:
y_pred=random_forest.predict(X_test)
random_forest.score(X_test,y_test)

0.7631296891747053

In [29]:
from sklearn.model_selection import cross_val_score
results = cross_val_score(estimator=random_forest, X=X_train, y=y_train, cv=5)

print(results)
print(results.mean())

[0.75672658 0.75926753 0.75848593 0.75748102 0.75424297]
0.75724080556984


### DecisionTreeClassifier

In [30]:
dt_classifier = DecisionTreeClassifier(max_depth = 12,random_state=42)
dt_classifier.fit(X_train, y_train)


DecisionTreeClassifier(max_depth=12, random_state=42)

In [31]:
y_pred=dt_classifier.predict(X_test)
dt_classifier.score(X_test,y_test)


results = cross_val_score(estimator=dt_classifier, X=X_train, y=y_train, cv=5)

print(results)
print(results.mean())

[0.72144691 0.72398392 0.72420724 0.72967843 0.71862439]
0.7235881767102941


### Naive Bayes

In [32]:
nb_classifier = naive_bayes.GaussianNB().fit(X_train, y_train)
y_pred = nb_classifier.predict(X_test)

results = cross_val_score(estimator=nb_classifier, X=X_train, y=y_train, cv=5)

print(results)
print(results.mean())

[0.57820699 0.58686914 0.56018312 0.60216615 0.59122376]
0.5837298301253027


### Linear Regresion

In [33]:
Linear = LinearRegression()
Linear.fit(X_train,y_train )

results = cross_val_score(estimator=Linear, X=X_train, y=y_train, cv=5)

print(results)
print(results.mean())

[0.23453434 0.22421473 0.25281855 0.23860308 0.23780025]
0.23759419114934982


### Prediccion

Finalmente hacemos la prediccion final para los datos 'Test_set_values'. Hago unas transfromaciones finales para presentar las predicciones juntos a su 'id' respectivo.

In [34]:

predicton = random_forest.predict(DF_test)
predicton = pd.DataFrame(predicton)
predicton.rename(columns={0:'status_group'},inplace=True)

In [35]:
pred1 = pd.merge(DF_test_id,predicton,left_index=True,right_index=True)
pred1.status_group.replace((0,1,2),('functional','functional needs repair','non functional'),inplace=True)
pred1 = pred1.set_index('id')

In [36]:
pred1.value_counts()

status_group           
functional                 8764
non functional             5368
functional needs repair     718
dtype: int64

# Conclusion:

Nuestra prediccion resulta:

**functional**: 8764
**non functional**: 5368
**functional needs repair**: 718

El 'accuracy' de la competencias Projecto Pump it Up: Data Mining the Water Table resulto en 0.7585 versus la prediccion de nuestro cross_val_score promedio fue 0.7572.